In [30]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph langchain-openai langchain-pinecone beautifulsoup4


In [31]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Introduce tu clave de API de LangSmith:")
os.environ["OPENAI_API_KEY"] = getpass.getpass("Introduce tu clave de API de OpenAI:")


Introduce tu clave de API de LangSmith:··········
Introduce tu clave de API de OpenAI:··········


In [32]:
from langchain.chat_models import init_chat_model
from langchain_openai import OpenAIEmbeddings

llm = init_chat_model("gpt-4o-mini", model_provider="openai")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


In [33]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

api_key = getpass.getpass("Introduce tu clave de API de Pinecone:")
pc = Pinecone(api_key=api_key)

index_name = "bmw-m235i-index"

index = pc.Index(index_name)
vector_store = PineconeVectorStore(embedding=embeddings, index=index)


Introduce tu clave de API de Pinecone:··········


In [43]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    web_paths=["https://extremamotor.es/prueba-bmw-m235i-coupe-326-cv-explosivos/"],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(name=["p", "div", "span", "h1", "h2", "h3"])
    ),
)
docs = loader.load()


In [44]:
print(f"🔹 Fragmentos generados: {len(all_splits)}")


🔹 Fragmentos generados: 8


In [36]:
_ = vector_store.add_documents(documents=all_splits)


In [37]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")


In [38]:
from langchain_core.documents import Document
from typing import List, TypedDict

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


In [39]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

question = "¿Cuáles son las especificaciones técnicas del BMW M235i?"
response = graph.invoke({"question": question})
print(response["answer"])


El BMW M235i cuenta con un motor N55 sobrealimentado de 326 caballos de potencia y puede acelerar de 0 a 100 km/h en 5 segundos con cambio manual, y en 4,8 segundos con la transmisión ZF de 8 velocidades. Es un coupé de tracción trasera, diseñado para ofrecer una experiencia de conducción deportiva. Aunque no es un M puro, tiene un ADN de M y destaca en su rendimiento.


In [40]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

question = "¿Cuanta potencia tiene el BMW M235i?"
response = graph.invoke({"question": question})
print(response["answer"])


El BMW M235i cuenta con un potente motor N55 que desarrolla 326 caballos de potencia. Esto le permite acelerar de 0 a 100 km/h en 5 segundos con cambio manual y en 4,8 segundos con cambio automático. Aunque no es un M puro, ofrece sensaciones deportivas destacadas.


In [41]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

question = "¿Que marca es el m235i?"
response = graph.invoke({"question": question})
print(response["answer"])


El M235i es un modelo de la marca BMW, específicamente de la serie 2 Coupé. Este vehículo se distingue por su diseño con el paquete M y un motor potente de 326 caballos. Aunque no es un modelo M puro, se acerca a sus características deportivas.
